In [1]:
import ase
from ase.visualize import view
from ase.io import read, write
import re
import os

In [20]:
def Hubbard_Input(file_name):
    def get_AB():
        # Regular expression to match elements (1, 2, or 3 letters, starting with a capital letter)
        elements = re.findall(r'[A-Z][a-z]?[a-z]?', file_name)
        
        # Remove 'O' for Oxygen from the list
        elements = [element for element in elements if element != 'O']
        
        # Ensure we have at least two elements
        if len(elements) >= 2:
            A_site = elements[0]
            B_site = elements[1]
        else:
            raise ValueError("File name does not contain enough elements to determine A Site and B Site")
        
        return A_site, B_site
    
    A, B = get_AB()
    
    def generate_upf_dict(folder_path):
        # List to hold the file names
        file_names = os.listdir(folder_path)
        
        # Dictionary to hold the element symbols and file names
        upf_dict = {}
        
        for file_name in file_names:
            # Use regex to extract the element symbol from the file name
            match = re.match(r'([a-zA-Z]+)', file_name)
            if match:
                element_symbol = match.group(1).capitalize()
                upf_dict[element_symbol] = file_name
        
        return upf_dict

    # Example usage
    folder_path = "SSSP_1.3.0_PBEsol_efficiency"
    upf_files = generate_upf_dict(folder_path)
    

    A_psuedo = upf_files[A]
    B_psuedo = upf_files[B]
    O_psuedo = upf_files['O']
    
    print(f"Elements {A}, {B}")
    print(f"A Pseudo: {A_psuedo}")
    print(f"B Pseudo: {B_psuedo}")
    print(f"O Pseudo: {O_psuedo}")

file_name = "BaTiO3.cif"
Hubbard_Input(file_name)


Elements Ba, Ti
A Pseudo: Ba.pbesol-spn-kjpaw_psl.1.0.0.UPF
B Pseudo: ti_pbesol_v1.4.uspp.F.UPF
O Pseudo: O.pbesol-n-kjpaw_psl.0.1.UPF


In [22]:
import re
import os

def Hubbard_Input(file_name, folder_path):
    def get_AB(file_name):
        # Regular expression to match elements (1, 2, or 3 letters, starting with a capital letter)
        elements = re.findall(r'[A-Z][a-z]?[a-z]?', file_name)
        
        # Remove 'O' for Oxygen from the list
        elements = [element for element in elements if element != 'O']
        
        # Ensure we have at least two elements
        if len(elements) >= 2:
            A_site = elements[0]
            B_site = elements[1]
        else:
            raise ValueError("File name does not contain enough elements to determine A Site and B Site")
        
        return A_site, B_site
    
    A, B = get_AB(file_name)
    
    def generate_upf_dict(folder_path):
        # List to hold the file names
        file_names = os.listdir(folder_path)
        
        # Dictionary to hold the element symbols and file names
        upf_dict = {}
        
        for file_name in file_names:
            # Use regex to extract the element symbol from the file name
            match = re.match(r'([a-zA-Z]+)', file_name)
            if match:
                element_symbol = match.group(1).capitalize()
                upf_dict[element_symbol] = file_name
        
        return upf_dict

    upf_files = generate_upf_dict(folder_path)
    
    A_psuedo = upf_files[A]
    B_psuedo = upf_files[B]
    O_psuedo = upf_files['O']
    
    print(f"Elements {A}, {B}")
    print(f"A Pseudo: {A_psuedo}")
    print(f"B Pseudo: {B_psuedo}")
    print(f"O Pseudo: {O_psuedo}")
    
    # Read original input file
    with open(file_name, 'r') as f:
        original_input = f.read()
    
    # Extract atomic positions
    atomic_positions_start = original_input.find('ATOMIC_POSITIONS angstrom') + len('ATOMIC_POSITIONS angstrom')
    atomic_positions_end = original_input.find('K_POINTS')
    atomic_positions_block = original_input[atomic_positions_start:atomic_positions_end].strip().splitlines()
    
    # Reorder atomic positions
    a_site_atoms = [pos for pos in atomic_positions_block if A in pos]
    b_site_atoms = [pos for pos in atomic_positions_block if B in pos]
    o_atoms = [pos for pos in atomic_positions_block if 'O' in pos and B not in pos]
    
    new_atomic_positions = b_site_atoms + o_atoms + a_site_atoms
    
    # Replace pseudopotentials in the input file
    new_input = re.sub(r'\bBa\.sol-spn-kjpaw_psl\.1\.0\.0\.UPF\b', f'Ba.{A_psuedo}', original_input)
    new_input = re.sub(r'\bO\.sol-n-kjpaw_psl\.0\.1\.UPF\b', f'O.{O_psuedo}', new_input)
    new_input = re.sub(r'\bti_sol_v1\.4\.uspp\.F\.UPF\b', f'Ti.{B_psuedo}', new_input)
    
    # Now, construct the Hubbard input part
    hubbard_input = f"\nHUBBARD {{ortho-atomic}}\nU {B}-3d 1.d-10\nV {B}-3d O-2p 1 2 1.d-10\n"
    
    # Append Hubbard input to the modified input file
    new_input += hubbard_input
    
    return new_input

# Example usage with a CIF file name and folder path containing UPF files
file_name = "BaTiO3.pwi"
folder_path = "SSSP_1.3.0_PBEsol_efficiency"
new_input = Hubbard_Input(file_name, folder_path)

print("\nGenerated Input with Hubbard Parameters:")
print(new_input)


Elements Ba, Ti
A Pseudo: Ba.pbesol-spn-kjpaw_psl.1.0.0.UPF
B Pseudo: ti_pbesol_v1.4.uspp.F.UPF
O Pseudo: O.pbesol-n-kjpaw_psl.0.1.UPF

Generated Input with Hubbard Parameters:
&CONTROL
  calculation = 'scf'
  etot_conv_thr =   5.0000000000d-05
  forc_conv_thr =   1.0000000000d-04
  max_seconds =   2.7360000000d+04
  outdir = './out/'
  prefix = 'aiida'
  pseudo_dir = './pseudo/'
  restart_mode = 'from_scratch'
  tprnfor = .true.
  tstress = .true.
  verbosity = 'high'
/
&SYSTEM
  degauss =   1.0000000000d-02
  ecutrho =   4.0000000000d+02
  ecutwfc =   5.0000000000d+01
  ibrav = 0
  nat = 5
  nbnd = 24
  nspin = 1
  ntyp = 3
  occupations = 'smearing'
  smearing = 'cold'
/
&ELECTRONS
  conv_thr =   1.0000000000d-09
  electron_maxstep = 80
  mixing_beta =   4.0000000000d-01
/
ATOMIC_SPECIES
Ba     137.327 Ba.pbesol-spn-kjpaw_psl.1.0.0.UPF
O      15.9994 O.pbesol-n-kjpaw_psl.0.1.UPF
Ti     47.867 ti_pbesol_v1.4.uspp.F.UPF
ATOMIC_POSITIONS angstrom 
Ti           0.0000000000       0.0000